# Comparative modeling of head and hinge regions of yeast Smc5/6
<br>
Tanmoy Sanyal<br>
Sali lab, UCSF

Necessary libraries:

In [38]:
import os
import numpy as np
import copy
import json
import pandas as pd ; pd.options.display.float_format = "{:,.2f}".format

from Bio import SeqIO
from Bio.PDB import PDBParser, PDBIO, Select, Structure, Model, Chain, Residue

File paths containing input data: sequence, crosslinks, comparative models automatically designed using SWISS-MODEL and comparative models designed using MODELLER.

In [55]:
INPUT_DATADIR = os.path.abspath("../data")
FASTA_FN = os.path.join(INPUT_DATADIR, "holocomplex.fasta.txt")
XL_FN = os.path.join(INPUT_DATADIR, "xl", "xl_all.csv")

SMC5_HINGE_DIR = os.path.join("smc5_hinge_SWISSMODEL")
SMC5_HEAD_DIR = os.path.join("smc56_head_MODELLER")

SMC6_HINGE_DIR = os.path.join("smc6_hinge_SWISSMODEL")
SMC6_HEAD_DIR = os.path.join("smc56_head_MODELLER")

Output directory for the best comparative models and directory containing all image files:

In [15]:
OUTDIR = "best_models"
os.makedirs(OUTDIR, exist_ok=True)

IMGDIR = "images"

Cutoffs for DSSO and CDI crosslinkers: <br>
(Cutoff for CDI crosslinker motivated from [The First Zero‐Length Mass Spectrometry‐Cleavable Cross‐Linker for Protein Structure Analysis, Hage, Iacobucci, Ang. Chem., 2017](https://onlinelibrary.wiley.com/doi/abs/10.1002/anie.201708273)

In [6]:
XL_CUTOFFS = {"DSSO": 30.0, "CDI": 20.0}

Helper function to extract specific regions from sequences:

In [7]:
def _extract_sequence_region(protein_name, fasta_id=None, start_residue=None, stop_residue=None):
    fasta = {r.id: r.seq._data for r in SeqIO.parse(FASTA_FN, format="fasta")}
    
    if fasta_id is None: fasta_id = protein_name
    seq = fasta[fasta_id]
    
    if start_residue is None: start_residue = 1
    if stop_residue is None: stop_residue = len(seq)
    print("region length = %d AA" % len(seq))
    print("pdb offset = %d" % (start_residue-1))
    print("sub-sequence: %s" % seq[(start_residue-1) : stop_residue])

Helper function to rank comparative models according to intra-molecular crosslink (XL) satisfaction:<br>
Note, in this function and in other functions later ```offset``` is how much you'd need to add to the index of a residue to get the same residue index as in the original fasta sequence for this molecule (smc5 or smc6).

In [59]:
def _rank_model_by_XL(pdb_fn, protein_name, offset=0):
    # extract intra-molecular XLs for this molecule
    df = pd.read_csv(XL_FN)
    xls = []
    for i in range(len(df)):
        p1, r1, p2, r2, linker = tuple(df.iloc[i])
        if p1 == p2 == protein_name:
            xls.append((p1, r1, p2, r2, linker, XL_CUTOFFS[linker]))
    
    # extract residue (CA) coordinates
    model = PDBParser(QUIET=True).get_structure("x", pdb_fn)[0]
    residues = {}
    for r in model.get_residues():
        key = r.id[1] + offset
        val = r["CA"]
        residues[key] = val
    
    # calculate XL satisfaction
    n_cov, n_sat, n_tot = 0, 0, len(df)
    
    for xl in xls:
        p1, r1, p2, r2, linker, cutoff = xl
        if not (r1 in residues and r2 in residues): continue
        n_cov += 1
        if residues[r1] - residues[r2] <= cutoff: n_sat += 1
    return n_sat, n_cov, n_tot

Helper function to get quality of comparative models from SWISSMODEL:

In [39]:
def _analyze_SWISSMODEL_quality(SWISSMODEL_output_dir, protein_name, nmodels=10, offset=0):
    quality = []
    for i in range(nmodels):
        model_dir = os.path.join(SWISSMODEL_output_dir, "model", "%02d" % (i+1))
        pdb_fn = os.path.join(model_dir, "model.pdb")
        
        # get XL stats
        n_sat, n_cov, n_tot = _rank_model_by_XL(pdb_fn, protein_name, offset)
        if n_cov == 0:
            frac_cov = 0
            frac_sat = 0
        else:
            frac_cov = float(n_cov) / n_tot
            frac_sat = float(n_sat) / n_cov
        
        # get resolution
        with open(os.path.join(model_dir, "report.json"), "r") as of:
            report = json.load(of)
        report = report["modelling"]
        template = report["pdb_id"] + "." + report["chain"]
        resolution = report["resolution"]
        seq_id = report["seq_id"]
        model_num = i+1
        
        this_quality = (model_num, template, resolution, seq_id, frac_cov*100, frac_sat*100, n_sat, n_cov)
        quality.append(this_quality)
    
    # organize into a pandas dataframe
    columns = ["model", "template", "resolution", "sequence_identity(%)", "XL_coverage(%)", "XL_satisfaction(%)", "num_XL_covered", "num_XL_satisfied"]
    df_quality = pd.DataFrame(quality, columns=columns)
    display(df_quality)

Helper function to extract the best model from a list of models returned from the SWISSMODEL webserver

In [44]:
def _get_SWISSMODEL_best(SWISSMODEL_output_dir, model_num, out_pdb_fn, chain_id="A", offset=0):
    in_pdb_fn = os.path.join(SWISSMODEL_output_dir, "model", "%02d" % model_num, "model.pdb")
    in_model = PDBParser().get_structure("x", in_pdb_fn)[0]
    
    model = Model.Model(0)
    chain = Chain.Chain(chain_id)
    for r in in_model.get_residues():
        if r.id[0] != " ": continue   # ignore HETATMS
        new_id = (r.id[0], r.id[1]+offset, r.id[2])
        new_r = Residue.Residue(id=new_id, resname=r.resname, segid=r.segid)
        [new_r.add(a) for a in r.get_atoms()]
        chain.add(new_r)
    model.add(chain)
    
    io = PDBIO()
    io.set_structure(model)
    io.save(os.path.join(OUTDIR, out_pdb_fn),
            preserve_atom_numbering=True)

## Smc5 Hinge (462, 639)

We extract the subsequence for the Smc5 hinge region and put it into [SWISSMODEL](https://swissmodel.expasy.org/) which automatically searches templates for us. Then we build models for templates that are crystal structures, have moderate to high resolution and sequence identity with the target subsequence. 

In [26]:
_extract_sequence_region(protein_name="smc5", fasta_id="smc5", start_residue=462, stop_residue=639)

region length = 1093 AA
pdb offset = 461
sub-sequence: LKEVRDAVLMVREHPEMKDKILEPPIMTVSAINAQFAAYLAQCVDYNTSKALTVVDSDSYKLFANPILDKFKVNLRELSSADTTPPVPAETVRDLGFEGYLSDFITGDKRVMKMLCQTSKIHTIPVSRRELTPAQIKKLITPRPNGKILFKRIIHGNRLVDIKQSAYGSKQVFPTDVS


Here are XL coverage and satisfaction statistics for the top 10 models returned from SWISSMODEL:

In [40]:
_analyze_SWISSMODEL_quality(SWISSMODEL_output_dir=SMC5_HINGE_DIR, protein_name="smc5", nmodels=10, offset=461)

,model,template,resolution,sequence_identity(%),XL_coverage(%),XL_satisfaction(%),num_XL_covered,num_XL_satisfied
0,1,5mg8.A,2.75,32.21,3.18,100.00,11,11
1,2,5mg8.A,2.75,32.21,3.18,100.00,11,11
2,3,3r64.A,2.57,28.85,0.00,0.00,0,0
3,4,4e4g.A,2.90,28.30,0.00,0.00,0,0
4,5,4e4g.B,2.90,28.30,0.00,0.00,0,0
5,6,4e4g.A,2.90,28.30,0.00,0.00,0,0
6,7,5mg8.A,2.75,27.54,3.76,92.31,12,13
7,8,5mg8.A,2.75,27.54,3.76,92.31,12,13
8,9,4dng.B,2.50,22.64,0.00,0.00,0,0
9,10,4dng.A,2.50,22.64,0.00,0.00,0,0


We prefer higher sequence identity (more reliable) over a larger number of satisfied XLs. This makes the model # 1 the best model over model numbers 6 or 7.

In [51]:
_get_SWISSMODEL_best(SWISSMODEL_output_dir=SMC5_HINGE_DIR, model_num=1, out_pdb_fn="smc5_hinge.pdb", chain_id="A", offset=461)

Comparative model of Smc5 hinge region: 462-639 @ 2.75 Å resolution from hinge domain of [Smc5 of fission yeast (*S.pombe*)](https://www.rcsb.org/structure/5mg8)<br>
![](images/smc5_hinge.png)

## Smc6 Hinge (504, 693)

Same as with the Smc5 hinge region, we extract the corresponding subsequence and use SWISSMODEL to derive a number of comparative models, which are then ranked according to XL satisfaction.

In [47]:
_extract_sequence_region(protein_name="smc6", fasta_id="smc6", start_residue=504, stop_residue=693)

region length = 1114 AA
pdb offset = 503
sub-sequence: DTFLMNFDRNMDRLLRTIEQRKNEFETPAIGPLGSLVTIRKGFEKWTRSIQRAISSSLNAFVVSNPKDNRLFRDIMRSCGIRSNIPIVTYCLSQFDYSKGRAHGNYPTIVDALEFSKPEIECLFVDLSRIERIVLIEDKNEARNFLQRNPVNVNMALSLRDRRSGFQLSGGYRLDTVTYQDKIRLKVNSS


In [49]:
_analyze_SWISSMODEL_quality(SWISSMODEL_output_dir=SMC6_HINGE_DIR, protein_name="smc6", nmodels=10, offset=503)

,model,template,resolution,sequence_identity(%),XL_coverage(%),XL_satisfaction(%),num_XL_covered,num_XL_satisfied
0,1,5mg8.B,2.75,31.45,0.87,100.00,3,3
1,2,5mg8.B,2.75,31.45,0.87,100.00,3,3
2,3,5mg8.B,2.75,28.02,2.60,88.89,8,9
3,4,2uvp.B,1.70,24.32,0.00,0.00,0,0
4,5,1gxl.A,3.00,16.44,0.87,100.00,3,3
5,6,1gxl.A,3.00,16.44,0.87,100.00,3,3
6,7,4rsj.A,3.50,14.52,0.29,100.00,1,1
7,8,1gxk.A,3.00,14.29,0.00,0.00,0,0
8,9,5mg8.A,2.75,12.99,2.31,100.00,8,8
9,10,6wg4.A,2.31,12.33,0.87,100.00,3,3


The choice here is clear. Model # 3 preserves a high sequence identity while also producing high XL satisfaction and coverage.

In [50]:
_get_SWISSMODEL_best(SWISSMODEL_output_dir=SMC6_HINGE_DIR, model_num=3, out_pdb_fn="smc6_hinge.pdb", chain_id="B", offset=503)

Comparative model of Smc6 hinge region: 504-693 @ 2.75 Å resolution from hinge domain of [Smc5 of fission yeast (*S.pombe*)](https://www.rcsb.org/structure/5mg8)<br>
![](images/smc6_hinge.png)

## Smc5/6 head region
Smc5  :  42-204 (head NTD), 949-1093 (head CTD)<br>
Smc6  :  80-232 (head NTD), 988-1114 (head CTD)<br>

Based on preliminary integrative models of Smc5 and Smc6, it was found that modeling the N- and C-terminal regions of the heads separately lead to the the heads being farther apart than usually found in Smc complexes. This resulted primarily from the lack of XL information in the head region. To enforce the heads to be closer, we'll develop a comparative model of the combined head region of Smc5/6, using a template that has Smc5 head region dimer/(s) in them. A good first choice is [6pqw](https://www.rcsb.org/structure/6QPW), which is a heterotrimeric complex of yeast cohesin (Smc1/3) and bacterial Scc1. Chain A of this complex (PDB ID: 6qpw) is Smc1 and chain C is Smc3.<br>

However, from a SWISSMODEL template search, it was found that there can be other contenders for template selection. E.g. 6qpw and [1xew](https://www.rcsb.org/structure/1XEW) (Smc complex from *P.furiosus*) both have high sequence identity with Smc5/6 head region. However 1xew has C terminal region of chain X and N terminal region of chain Y missing, while 6qpw has C terminal region of Smc1 missing. These gaps can in principle be fulfilled by a very recently deposited structure of human condensin [6zz6](https://www.rcsb.org/structure/6ZZ6) which has average 21-24% sequence identity with Smc5/6 head region. So the ideal scenario would be to build a comparative model from all three of these templates, taking the missing regions as much as posible from the more similar 6qpw and the remaining regions from 6zz6.<br>

But first, we will perform several combinations of alignments to figure out which chain in each template corresponds more to Smc5 and which chain more to Smc6. Pairwise alignments and MSAs were generated using the [Clustal Omega webserver](https://toolkit.tuebingen.mpg.de/tools/clustalo). The seqeunce identities for various combinations are as:<br><br>

6ZZ6<br>
Smc5 head and 6zz6.A have 66 equivalences, and are 21.43% identical<br>
Smc6 head and 6zz6.B have 63 equivalences, and are 22.50% identical<br>
Smc5 head and 6zz6.B have 65 equivalences, and are 21.10% identical<br>
Smc6 head and 6zz6.A have 69 equivalences, and are 24.64% identical<br>
<br>

6QPW<br>
Smc5 head and 6qpw.A have 42 equivalences, and are 25.77% identical<br>
Smc6 head and 6qpw.C have 66 equivalences, and are 23.57% identical<br>
Smc5 head and 6qpw.C have 68 equivalences, and are 22.08% identical<br>
Smc6 head NTD and 6qpw.A have 36 equivalences, and are 23.53% identical<br>
<br>

1XEW<br>
Smc5 head NTD and 1xew.X have 35 equivalences, and are 23.81% identical<br>
Smc6 head CTD and 1xew.Y have 31 equivalences, and are 24.41% identical<br>
Smc5 head CTD and 1xew.Y have 33 equivalences, and are 22.76% identical<br>
Smc6 head NTD and 1xew.X have 32 equivalences, and are 21.77% identical<br>
<br>

Based on these, the optimal template combination that assures higher sequence identitites, seems to be: 6qpw.A for Smc5 head NTD, 6zz6.A for Smc5 head CTD_termina, and 6qpwC for both NTD and CTD of Smc6 head. But, to keep things simpler, we will use just 6qpw and 6zz6. Clustal Omega is again used to build a MSA from Smc5/6 head region, 6qpw (chains A & C) and 6zz6 (chains A & B) which is then served as an input to [MODELLER.](https://salilab.org/modeller/)<br>

The MODELLER python script, template pdb files and alignment file containing the MSA are kept in the directory ```smc56_head_MODELLER```. The entire MODELLER output is kept in ```smc56_head_MODELLER/output``` and includes 20 alternate models (pdb files) generated by MODELLER. We will rank these models by XL satisfaction. 

In [61]:
head_quality = []

# parse the XL file
df = pd.read_csv(XL_FN)
xls = []
for i in range(len(df)):
    p1, r1, p2, r2, linker = tuple(df.iloc[i])
    xls.append((p1, r1, p2, r2, linker, XL_CUTOFFS[linker]))

# check XL satisfaction for all comparative models built by MODELLER
for i in range(20):
    # read the pdb file
    pdb_fn = os.path.join(SMC5_HEAD_DIR, "output", "smc56_head.B9999%04d.pdb" % (i+1))
    model = PDBParser(QUIET=True).get_structure("x", pdb_fn)[0]
    
    # store CA coordinates of all residues of Smc5 and Smc6 head regions
    smc5_res, smc6_res = {}, {}
    for r in list(model["A"].get_residues()) + list(model["B"].get_residues()):
        smc5_res[r.id[1]] = r["CA"]
    for r in list(model["C"].get_residues()) + list(model["D"].get_residues()):
        smc6_res[r.id[1]] = r["CA"]
        
    nsat, ncov, frac_sat = 0.0, 0.0, 0.0
    for xl in xls:
        p1, r1, p2, r2, linker, cutoff = xl
        a1, a2 = None, None
        
        if (p1=="smc5" and r1 in smc5_res):
            a1 = smc5_res[r1]
        elif (p1=="smc6" and r1 in smc6_res):
            a1 = smc6_res[r1]
        
        if (p2=="smc5" and r2 in smc5_res):
            a2 = smc5_res[r2]
        elif (p2=="smc6" and r2 in smc6_res):
            a2 = smc6_res[r2]
        
        if (a1 is None) or (a2 is None): continue
        ncov += 1
        if a1-a2 <= cutoff: nsat += 1
    
    if nsat > 0: frac_sat = nsat / ncov
    head_quality.append((i+1, nsat, ncov, frac_sat))

columns = ["model_num", "num_XL_satisfied", "num_XL_covered", "XL_satisfaction(%)"]
df_quality = pd.DataFrame(head_quality, columns=columns)
display(df_quality)

,model_num,num_XL_satisfied,num_XL_covered,XL_satisfaction(%)
0,1,14.00,14.00,1.00
1,2,13.00,14.00,0.93
2,3,14.00,14.00,1.00
3,4,14.00,14.00,1.00
4,5,14.00,14.00,1.00
5,6,13.00,14.00,0.93
6,7,14.00,14.00,1.00
7,8,13.00,14.00,0.93
8,9,13.00,14.00,0.93
9,10,14.00,14.00,1.00


Since all the XL satisfactions are very close, we'll select the model with the lowest DOPE score. The DOPE scores can be found at the end of the file ```smc56_head_MODELLER/output/log.txt```. Model # 18 has the lowest DOPE score. We'll select this as the comparative model of the Smc5/6 head region. Chains A+B represent the Smc5 head and chains C+D represent the Smc6 head. The two Smc heads will be treated as two distinct rigid bodies during integrative modeling. This model is copied to ```best_models``` directory and stored as ```smc56_head-dimer.pdb```.

Comparative model of Smc5/6 combined head region: Smc5:42-204, 949-1093 and Smc6:80-232, 988-1114 from a combination of templates [6qpw](https://www.rcsb.org/structure/6QPW) (chains A & C) @ 3.30 Å and [6zz6](https://www.rcsb.org/structure/6ZZ6) (chains A & B) @ 3.40 Å.<br>
Smc5 is in red and Smc6 is in blue. <br>
![](images/smc56_head.png)